In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

from model import Model


# -------------------
# Smoke test
# -------------------
model = Model()

out = model._compute_updated_ratings(
    1,
    1,
    True,
    np.array([0.0, 0.0]),
    np.array([[1.0, 0.5], [0.5, 1.0]]),
    np.array([0.0, 0.0]),
    np.array([[1.0, 0.5], [0.5, 1.0]]),
    # max_steps=1000,
    # tol_grad=1e-6,
    # lm_damping=1e-2,
)

print(out[0], "\n", out[1], "\n", out[2], "\n", out[3])
print(out[4])

[-0.00130145  0.00130145] 
 [[0.65651448 0.28135179]
 [0.28135179 0.65651448]] 
 [-0.00130145  0.00130145] 
 [[0.65651448 0.28135179]
 [0.28135179 0.65651448]]
{'converged': True, 'iters': 4, 'f': 1.9942978168568484, 'grad_norm': 6.327851629055886e-08, 'grad_norm_recomputed': 6.327851629055886e-08, 'theta_map': array([-0.00130145,  0.00130145, -0.00130145,  0.00130145])}
